**Team Members**
* Shubh Todi 		19BCE1823
* Shiva Rajan 		19BCE1471
* Satwik Samyaan 	19BCE1220
* Abhishek N N 		20BCE1025
 
 
Code of Academic Integrity
 
We affirm that
This work is our own original work and is not a borrowed work, either from other students or from assignments for other courses.
We have not given or received any unauthorized help on this assignment.

This submission is free from

                      - Plagiarism

                      - Fabrication of facts

                      - Unauthorized assistance

                      - Collusion


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.shape

(404290, 6)

In [4]:
new_df = df.sample(30000,random_state=0)
# new_df=df.copy()

In [5]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
187265,187265,61254,82970,Are we near World War 3?,Is World War 3 closer than it has ever been?,1
6557,6557,12839,12840,What are the best Doctor Doom stories?,What are the best comics featuring Doctor Doom?,1
139264,139264,221574,221575,Why are Newton's rings circular in shape? Why ...,Why is a circular ring pattern obtained in New...,0
240135,240135,227307,351892,How does Wikipedia ensure content quality / au...,How does Wikipedia keep content quality high?,1
3336,3336,6613,6614,Height: How would a 14 year old increase his h...,How should I increase my height?,1


In [6]:
def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    
    return q
    

In [7]:
preprocess("I've already! wasn't <b>done</b>?")

'i have already  was not done'

In [8]:
new_df['question1'] = new_df['question1'].apply(preprocess)
new_df['question2'] = new_df['question2'].apply(preprocess)

In [9]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
187265,187265,61254,82970,are we near world war 3,is world war 3 closer than it has ever been,1
6557,6557,12839,12840,what are the best doctor doom stories,what are the best comics featuring doctor doom,1
139264,139264,221574,221575,why are newton s rings circular in shape why ...,why is a circular ring pattern obtained in new...,0
240135,240135,227307,351892,how does wikipedia ensure content quality au...,how does wikipedia keep content quality high,1
3336,3336,6613,6614,height how would a 14 year old increase his h...,how should i increase my height,1


In [10]:
new_df['q1_len'] = new_df['question1'].str.len() 
new_df['q2_len'] = new_df['question2'].str.len()

In [11]:
new_df['q1_num_words'] = new_df['question1'].apply(lambda row: len(row.split(" ")))
new_df['q2_num_words'] = new_df['question2'].apply(lambda row: len(row.split(" ")))
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words
187265,187265,61254,82970,are we near world war 3,is world war 3 closer than it has ever been,1,23,43,6,10
6557,6557,12839,12840,what are the best doctor doom stories,what are the best comics featuring doctor doom,1,37,46,7,8
139264,139264,221574,221575,why are newton s rings circular in shape why ...,why is a circular ring pattern obtained in new...,0,75,63,16,12
240135,240135,227307,351892,how does wikipedia ensure content quality au...,how does wikipedia keep content quality high,1,70,44,11,7
3336,3336,6613,6614,height how would a 14 year old increase his h...,how should i increase my height,1,51,31,11,6


In [12]:
def common_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return len(w1 & w2)

In [13]:
new_df['word_common'] = new_df.apply(common_words, axis=1)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common
187265,187265,61254,82970,are we near world war 3,is world war 3 closer than it has ever been,1,23,43,6,10,3
6557,6557,12839,12840,what are the best doctor doom stories,what are the best comics featuring doctor doom,1,37,46,7,8,6
139264,139264,221574,221575,why are newton s rings circular in shape why ...,why is a circular ring pattern obtained in new...,0,75,63,16,12,5
240135,240135,227307,351892,how does wikipedia ensure content quality au...,how does wikipedia keep content quality high,1,70,44,11,7,5
3336,3336,6613,6614,height how would a 14 year old increase his h...,how should i increase my height,1,51,31,11,6,3


In [14]:
def total_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return (len(w1) + len(w2))

In [15]:
new_df['word_total'] = new_df.apply(total_words, axis=1)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total
187265,187265,61254,82970,are we near world war 3,is world war 3 closer than it has ever been,1,23,43,6,10,3,16
6557,6557,12839,12840,what are the best doctor doom stories,what are the best comics featuring doctor doom,1,37,46,7,8,6,15
139264,139264,221574,221575,why are newton s rings circular in shape why ...,why is a circular ring pattern obtained in new...,0,75,63,16,12,5,25
240135,240135,227307,351892,how does wikipedia ensure content quality au...,how does wikipedia keep content quality high,1,70,44,11,7,5,17
3336,3336,6613,6614,height how would a 14 year old increase his h...,how should i increase my height,1,51,31,11,6,3,16


In [16]:
new_df['word_share'] = round(new_df['word_common']/new_df['word_total'],2)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share
187265,187265,61254,82970,are we near world war 3,is world war 3 closer than it has ever been,1,23,43,6,10,3,16,0.19
6557,6557,12839,12840,what are the best doctor doom stories,what are the best comics featuring doctor doom,1,37,46,7,8,6,15,0.40
139264,139264,221574,221575,why are newton s rings circular in shape why ...,why is a circular ring pattern obtained in new...,0,75,63,16,12,5,25,0.20
240135,240135,227307,351892,how does wikipedia ensure content quality au...,how does wikipedia keep content quality high,1,70,44,11,7,5,17,0.29
3336,3336,6613,6614,height how would a 14 year old increase his h...,how should i increase my height,1,51,31,11,6,3,16,0.19


In [17]:
# Advanced Features
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def fetch_token_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    SAFE_DIV = 0.0001 

    STOP_WORDS = stopwords.words("english")
    
    token_features = [0.0]*8
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    return token_features


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/Abhishek_N_N_20BCE1025/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
token_features = new_df.apply(fetch_token_features, axis=1)

new_df["cwc_min"]       = list(map(lambda x: x[0], token_features))
new_df["cwc_max"]       = list(map(lambda x: x[1], token_features))
new_df["csc_min"]       = list(map(lambda x: x[2], token_features))
new_df["csc_max"]       = list(map(lambda x: x[3], token_features))
new_df["ctc_min"]       = list(map(lambda x: x[4], token_features))
new_df["ctc_max"]       = list(map(lambda x: x[5], token_features))
new_df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
new_df["first_word_eq"] = list(map(lambda x: x[7], token_features))

In [19]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,word_total,word_share,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq
187265,187265,61254,82970,are we near world war 3,is world war 3 closer than it has ever been,1,23,43,6,10,...,16,0.19,0.749981,0.599988,0.000000,0.000000,0.499992,0.299997,0.0,0.0
6557,6557,12839,12840,what are the best doctor doom stories,what are the best comics featuring doctor doom,1,37,46,7,8,...,15,0.40,0.749981,0.599988,0.999967,0.999967,0.857131,0.749991,0.0,1.0
139264,139264,221574,221575,why are newton s rings circular in shape why ...,why is a circular ring pattern obtained in new...,0,75,63,16,12,...,25,0.20,0.399992,0.333328,0.599988,0.374995,0.416663,0.333331,0.0,1.0
240135,240135,227307,351892,how does wikipedia ensure content quality au...,how does wikipedia keep content quality high,1,70,44,11,7,...,17,0.29,0.599988,0.499992,0.999950,0.666644,0.714276,0.555549,0.0,1.0
3336,3336,6613,6614,height how would a 14 year old increase his h...,how should i increase my height,1,51,31,11,6,...,16,0.19,0.999950,0.333328,0.333322,0.249994,0.499992,0.299997,1.0,0.0


In [20]:
from array import array


def lcsubstrings(seq1, seq2, positions=False):
	L1, L2 = len(seq1), len(seq2)
	ms = []
	mlen = last = 0
	if L1 < L2:
		seq1, seq2 = seq2, seq1
		L1, L2 = L2, L1
	
	column = array('L', range(L2))
	
	for i in range(L1):
		for j in range(L2):
			old = column[j]
			if seq1[i] == seq2[j]:
				if i == 0 or j == 0:
					column[j] = 1
				else:
					column[j] = last + 1
				if column[j] > mlen:
					mlen = column[j]
					ms = [(i, j)]
				elif column[j] == mlen:
					ms.append((i, j))
			else:
				column[j] = 0
			last = old
	
	if positions:
		return (mlen, tuple((i - mlen + 1, j - mlen + 1) for i, j in ms if ms))
	return set(seq1[i - mlen + 1:i + 1] for i, _ in ms if ms)

In [21]:
# import distance

def fetch_length_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    length_features = [0.0,0.0,0.0]
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features
    
    # Absolute length features
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens))/2
    
    strs = list(lcsubstrings(q1, q2))
    print(strs)
    length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)
    
    return length_features
    

In [22]:
length_features = new_df.apply(fetch_length_features, axis=1)

new_df['abs_len_diff'] = list(map(lambda x: x[0], length_features))
new_df['mean_len'] = list(map(lambda x: x[1], length_features))
new_df['longest_substr_ratio'] = list(map(lambda x: x[2], length_features))

[' world war 3']
['what are the best ']
[' newton s ring']
['how does wikipedia ']
[' increase ']
['how many p']
[' masood azhar', 's china block']
[' i forget ']
['what is the sahara  and how do the average temperatures there compare to the ones in the ']
['what is the most romantic thing ']
[' mutual fund']
['t time ']
[' in india  ', ' cantonment']
['ian women', ' iranian ']
['how does the structure of the education system in south america compare to the edcuation system in ']
['why do humans have 5 fingers']
[' indian constitution ']
['ter than ']
[' international recruit new grads from  what majors are they looking for']
[' knowledge base']
[' eurovision song contest']
['ment of mr  ajay chandrakar as a chhattisgarh ']
['how does balaji vishwanathan ']
[' midbrain activation ']
[' vatican ', ' country ']
[' carbohydrates ']
[' do ']
['is a passport ']
['e against dollar', ' demonetization ']
[' daenerys targaryen']
['e 2 ']
[' su', 'le ']
[' companies ']
[' should be legal']
[' co

In [23]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio
187265,187265,61254,82970,are we near world war 3,is world war 3 closer than it has ever been,1,23,43,6,10,...,0.599988,0.000000,0.000000,0.499992,0.299997,0.0,0.0,4.0,8.0,0.500000
6557,6557,12839,12840,what are the best doctor doom stories,what are the best comics featuring doctor doom,1,37,46,7,8,...,0.599988,0.999967,0.999967,0.857131,0.749991,0.0,1.0,1.0,7.5,0.473684
139264,139264,221574,221575,why are newton s rings circular in shape why ...,why is a circular ring pattern obtained in new...,0,75,63,16,12,...,0.333328,0.599988,0.374995,0.416663,0.333331,0.0,1.0,3.0,13.5,0.218750
240135,240135,227307,351892,how does wikipedia ensure content quality au...,how does wikipedia keep content quality high,1,70,44,11,7,...,0.499992,0.999950,0.666644,0.714276,0.555549,0.0,1.0,2.0,8.0,0.422222
3336,3336,6613,6614,height how would a 14 year old increase his h...,how should i increase my height,1,51,31,11,6,...,0.333328,0.333322,0.249994,0.499992,0.299997,1.0,0.0,4.0,8.0,0.312500


In [24]:
!pip3 install fuzzywuzzy

In [25]:
# Fuzzy Features
from fuzzywuzzy import fuzz

def fetch_fuzzy_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    fuzzy_features = [0.0]*4
    
    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features

In [26]:
fuzzy_features = new_df.apply(fetch_fuzzy_features, axis=1)

# Creating new feature columns for fuzzy features
new_df['fuzz_ratio'] = list(map(lambda x: x[0], fuzzy_features))
new_df['fuzz_partial_ratio'] = list(map(lambda x: x[1], fuzzy_features))
new_df['token_sort_ratio'] = list(map(lambda x: x[2], fuzzy_features))
new_df['token_set_ratio'] = list(map(lambda x: x[3], fuzzy_features))

In [27]:
print(new_df.shape)
new_df.head()

(30000, 28)


,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
187265,187265,61254,82970,are we near world war 3,is world war 3 closer than it has ever been,1,23,43,6,10,...,0.299997,0.0,0.0,4.0,8.0,0.500000,36,52,45,65
6557,6557,12839,12840,what are the best doctor doom stories,what are the best comics featuring doctor doom,1,37,46,7,8,...,0.749991,0.0,1.0,1.0,7.5,0.473684,70,68,80,88
139264,139264,221574,221575,why are newton s rings circular in shape why ...,why is a circular ring pattern obtained in new...,0,75,63,16,12,...,0.333331,0.0,1.0,3.0,13.5,0.218750,36,41,54,66
240135,240135,227307,351892,how does wikipedia ensure content quality au...,how does wikipedia keep content quality high,1,70,44,11,7,...,0.555549,0.0,1.0,2.0,8.0,0.422222,70,86,66,87
3336,3336,6613,6614,height how would a 14 year old increase his h...,how should i increase my height,1,51,31,11,6,...,0.299997,1.0,0.0,4.0,8.0,0.312500,63,65,62,76


In [28]:
# sns.pairplot(new_df[['ctc_min', 'cwc_min', 'csc_min', 'is_duplicate']],hue='is_duplicate')

In [29]:
# sns.pairplot(new_df[['ctc_max', 'cwc_max', 'csc_max', 'is_duplicate']],hue='is_duplicate')

In [30]:
# sns.pairplot(new_df[['last_word_eq', 'first_word_eq', 'is_duplicate']],hue='is_duplicate')

In [31]:
# sns.pairplot(new_df[['mean_len', 'abs_len_diff','longest_substr_ratio', 'is_duplicate']],hue='is_duplicate')

In [32]:
# sns.pairplot(new_df[['fuzz_ratio', 'fuzz_partial_ratio','token_sort_ratio','token_set_ratio', 'is_duplicate']],hue='is_duplicate')

In [33]:
# Using TSNE for Dimentionality reduction for 15 Features(Generated after cleaning the data) to 3 dimention

from sklearn.preprocessing import MinMaxScaler

X = MinMaxScaler().fit_transform(new_df[['cwc_min', 'cwc_max', 'csc_min', 'csc_max' , 'ctc_min' , 'ctc_max' , 'last_word_eq', 'first_word_eq' , 'abs_len_diff' , 'mean_len' , 'token_set_ratio' , 'token_sort_ratio' ,  'fuzz_ratio' , 'fuzz_partial_ratio' , 'longest_substr_ratio']])
y = new_df['is_duplicate'].values

In [34]:
# from sklearn.manifold import TSNE

# tsne2d = TSNE(
#     n_components=2,
#     init='random', # pca
#     random_state=101,
#     method='barnes_hut',
#     n_iter=1000,
#     verbose=2,
#     angle=0.5
# ).fit_transform(X)

In [35]:
# x_df = pd.DataFrame({'x':tsne2d[:,0], 'y':tsne2d[:,1] ,'label':y})

# draw the plot in appropriate place in the grid
# sns.lmplot(data=x_df, x='x', y='y', hue='label', fit_reg=False, size=8,palette="Set1",markers=['s','o'])


In [36]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
187265,are we near world war 3,is world war 3 closer than it has ever been
6557,what are the best doctor doom stories,what are the best comics featuring doctor doom
139264,why are newton s rings circular in shape why ...,why is a circular ring pattern obtained in new...
240135,how does wikipedia ensure content quality au...,how does wikipedia keep content quality high
3336,height how would a 14 year old increase his h...,how should i increase my height


In [37]:
final_df = new_df.drop(columns=['id','qid1','qid2','question1','question2'])
print(final_df.shape)
final_df.head()

(30000, 23)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
187265,1,23,43,6,10,3,16,0.19,0.749981,0.599988,...,0.299997,0.0,0.0,4.0,8.0,0.500000,36,52,45,65
6557,1,37,46,7,8,6,15,0.40,0.749981,0.599988,...,0.749991,0.0,1.0,1.0,7.5,0.473684,70,68,80,88
139264,0,75,63,16,12,5,25,0.20,0.399992,0.333328,...,0.333331,0.0,1.0,3.0,13.5,0.218750,36,41,54,66
240135,1,70,44,11,7,5,17,0.29,0.599988,0.499992,...,0.555549,0.0,1.0,2.0,8.0,0.422222,70,86,66,87
3336,1,51,31,11,6,3,16,0.19,0.999950,0.333328,...,0.299997,1.0,0.0,4.0,8.0,0.312500,63,65,62,76


In [38]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [39]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [40]:
final_df = pd.concat([final_df, temp_df], axis=1)
print(final_df.shape)
final_df.head()

(30000, 6023)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
187265,1,23,43,6,10,3,16,0.19,0.749981,0.599988,...,0,0,0,0,0,0,0,0,0,0
6557,1,37,46,7,8,6,15,0.40,0.749981,0.599988,...,0,0,0,0,0,0,0,0,0,0
139264,0,75,63,16,12,5,25,0.20,0.399992,0.333328,...,0,0,0,0,0,0,0,0,0,0
240135,1,70,44,11,7,5,17,0.29,0.599988,0.499992,...,0,0,0,0,0,0,0,0,0,0
3336,1,51,31,11,6,3,16,0.19,0.999950,0.333328,...,0,0,0,0,0,0,0,0,0,0


In [41]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(final_df.iloc[:,1:].values,final_df.iloc[:,0].values,test_size=0.2,random_state=1)

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7786666666666666

In [43]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred1 = xgb.predict(X_test)
accuracy_score(y_test,y_pred1)

0.787

In [44]:
from sklearn.metrics import confusion_matrix

In [45]:
# for random forest model
confusion_matrix(y_test,y_pred)

array([[3169,  577],
       [ 751, 1503]])

In [46]:
# for xgboost model
confusion_matrix(y_test,y_pred1)

array([[3123,  623],
       [ 655, 1599]])

In [47]:
def test_common_words(q1,q2):
    w1 = set(map(lambda word: word.lower().strip(), q1.split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), q2.split(" ")))    
    return len(w1 & w2)

In [48]:
def test_total_words(q1,q2):
    w1 = set(map(lambda word: word.lower().strip(), q1.split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), q2.split(" ")))    
    return (len(w1) + len(w2))

In [49]:
def test_fetch_token_features(q1,q2):
    
    SAFE_DIV = 0.0001 

    STOP_WORDS = stopwords.words("english")
    
    token_features = [0.0]*8
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    return token_features


In [50]:
def test_fetch_length_features(q1,q2):
    
    length_features = [0.0]*3
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features
    
    # Absolute length features
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens))/2
    
    strs = list(lcsubstrings(q1, q2))
    length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)
    
    return length_features

In [51]:
def test_fetch_fuzzy_features(q1,q2):
    
    fuzzy_features = [0.0]*4
    
    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features

In [52]:
def query_point_creator(q1,q2):
    
    input_query = []
    
    # preprocess
    q1 = preprocess(q1)
    q2 = preprocess(q2)
    
    # fetch basic features
    input_query.append(len(q1))
    input_query.append(len(q2))
    
    input_query.append(len(q1.split(" ")))
    input_query.append(len(q2.split(" ")))
    
    input_query.append(test_common_words(q1,q2))
    input_query.append(test_total_words(q1,q2))
    input_query.append(round(test_common_words(q1,q2)/test_total_words(q1,q2),2))
    
    # fetch token features
    token_features = test_fetch_token_features(q1,q2)
    input_query.extend(token_features)
    
    # fetch length based features
    length_features = test_fetch_length_features(q1,q2)
    input_query.extend(length_features)
    
    # fetch fuzzy features
    fuzzy_features = test_fetch_fuzzy_features(q1,q2)
    input_query.extend(fuzzy_features)
    
    # bow feature for q1
    q1_bow = cv.transform([q1]).toarray()
    
    # bow feature for q2
    q2_bow = cv.transform([q2]).toarray()
    
    
    
    return np.hstack((np.array(input_query).reshape(1,22),q1_bow,q2_bow))

In [53]:
q1 = 'Where is the capital of India?'
q2 = 'What is the current capital of Pakistan?'
q3 = 'Which city serves as the capital of India?'
q4 = 'What is the business capital of India?'

In [54]:
rf.predict(query_point_creator(q1,q2))

array([0])

In [63]:
from joblib import dump
dump(rf, 'model.joblib') 
dump(cv,"countvectorizer.joblib")

['model.joblib']

['countvectorizer.joblib']

---
---
---
---
---

In [56]:
from joblib import load
clf = load('model.joblib') 

In [57]:
q1 = 'Where is the capital of India?'
q2 = 'What is the current capital of Pakistan?'
q3 = 'Which city serves as the capital of India?'

In [58]:
print(clf.predict(query_point_creator(q1,q2)))
print(clf.predict(query_point_creator(q1,q3)))

[0]
[1]


In [59]:
q4='is sklearn and tenserflow same'
q5='is tenserflow and sklearn same'

In [60]:
print(clf.predict(query_point_creator(q4,q5)))

[1]


In [61]:
q6='should i doo sports or reading'
q7='please tell me what should i doo reading or sports'
q8='please tell me what should i doo engineering or medical'

In [67]:
print(clf.predict(query_point_creator(q6,q7)))
print(clf.predict(query_point_creator(q6,q8)))
print(clf.predict(query_point_creator(q7,q8)))

[1]
[0]
[0]
